In [8]:
#Import dependencies
import pandas as pd 
import numpy as np 
from sqlalchemy import create_engine, func
import psycopg2
from config import pwd, db_pwd

In [9]:
#Read in CSVs
diabetes_df = pd.read_csv('resources/cleaned_csvs/diabetes_state_cleaned.csv')
obesity_df = pd.read_csv('resources/cleaned_csvs/obesity_state_cleaned.csv')
overdose_df = pd.read_csv('resources/cleaned_csvs/overdose_state_cleaned.csv')
waitlist_df = pd.read_csv('resources/cleaned_csvs/kidney_waitlist_additions_cleaned.csv')

In [3]:
#Check csv reads
diabetes_df.head()

,Unnamed: 0,State,1995,1996,1997,1998,1999,2000,2001,2002,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,0,Alabama,0.05,0.06,0.06,0.07,0.08,0.08,0.08,0.09,...,0.12,0.12,0.12,0.14,0.13,0.14,0.15,0.14,0.15,0.14
1,1,Alaska,0.04,0.04,0.04,0.04,0.04,0.04,0.05,0.05,...,0.06,0.08,0.07,0.07,0.07,0.08,0.08,0.07,0.09,0.07
2,2,Arizona,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.06,...,0.09,0.10,0.11,0.11,0.10,0.10,0.11,0.10,0.11,0.11
3,3,Arkansas,0.06,0.06,0.06,0.07,0.07,0.07,0.07,0.07,...,0.10,0.11,0.11,0.12,0.13,0.13,0.14,0.12,0.14,0.14
4,4,California,0.05,0.05,0.06,0.06,0.07,0.08,0.07,0.07,...,0.09,0.09,0.10,0.10,0.10,0.10,0.10,0.11,0.10,0.10


In [4]:
obesity_df.head()

,Unnamed: 0,State,1995,1996,1997,1998,1999,2000,2001,2002,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,0,Alaska,0.157,0.167,0.177,0.188,0.198,0.209,0.2177,0.2263,...,0.259,0.274,0.257,0.284,0.297,0.298,0.314,0.342,0.295,0.305
1,1,North Carolina,0.163,0.172,0.181,0.190,0.199,0.209,0.2193,0.2297,...,0.294,0.291,0.296,0.294,0.297,0.301,0.318,0.321,0.330,0.340
2,2,Alabama,0.157,0.170,0.184,0.198,0.212,0.226,0.2453,0.2647,...,0.323,0.320,0.330,0.324,0.335,0.356,0.357,0.363,0.362,0.361
3,3,North Dakota,0.152,0.162,0.173,0.183,0.194,0.205,0.2157,0.2263,...,0.280,0.278,0.297,0.310,0.322,0.310,0.319,0.332,0.351,0.348
4,4,Arkansas,0.170,0.179,0.189,0.199,0.209,0.219,0.2300,0.2410,...,0.306,0.309,0.345,0.346,0.359,0.345,0.357,0.350,0.371,0.374


In [5]:
overdose_df.head()

,Unnamed: 0,State,1999,2000,2001,2002,2003,2004,2005,2006,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,0,Alabama,0.9,1.0,1.5,1.7,1.2,2.0,1.8,2.9,...,4.4,4.1,3.7,3.6,3.6,5.8,15.7,16.2,18.0,16.6
1,1,Alaska,4.5,4.3,NaN,NaN,NaN,NaN,11.4,4.3,...,13.4,10.0,10.0,11.1,9.4,10.7,16.0,16.8,20.2,14.6
2,2,Arizona,5.0,4.9,5.2,6.3,6.4,6.4,6.7,7.9,...,9.9,10.2,9.2,8.6,8.2,9.2,19.0,20.3,22.2,23.8
3,3,Arkansas,1.1,0.7,1.1,3.5,3.6,4.8,4.6,5.3,...,7.6,6.8,6.0,6.1,5.9,6.0,13.8,14.0,15.5,15.7
4,4,California,4.8,3.3,1.8,4.5,4.3,4.3,4.1,4.4,...,5.8,5.5,5.5,4.9,5.4,5.6,11.3,11.2,11.7,12.8


In [12]:
waitlist_df.head()

,Unnamed: 0,State,ABO,Ethnicity,To_Date,2020,2019,2018,2017,2016,...,2004,2003,2002,2001,2000,1999,1998,1997,1996,1995
0,0,All Center States,All ABO,All Ethnicities,634539,33043,41105,38799,35593,35417,...,27274,24617,23644,22497,22378,21835,19473,18405,17732,17257
1,1,All Center States,All ABO,White,301516,13937,17317,16378,15661,15530,...,13145,11989,11836,11441,11709,11952,10449,9845,9760,9510
2,2,All Center States,All ABO,Black,177840,9371,11804,10925,9877,10026,...,7605,6960,6642,6376,6256,5914,5504,5180,5073,4760
3,3,All Center States,All ABO,Hispanic,107669,6485,7976,7529,6634,6584,...,4260,3881,3382,3047,2882,2594,2232,2161,1784,1818
4,4,All Center States,All ABO,Unknown,3,0,0,0,0,0,...,0,0,1,0,0,0,0,2,0,0
